In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data
from lvmdrp.core import rss, image, sky
from lvmdrp.functions import skyMethod
from multiprocessing import Pool, cpu_count


def matchRes(in_rss, new_wave=None, new_lsf=None, resample_method="linear", err_sim=500, replace_error=1e10, parallel="auto"):

    if new_wave is None and new_lsf is None:
        return in_rss
    
    if parallel=='auto':
        cpus = cpu_count()
    else:
        cpus = int(parallel)
        
    # resample RSS to reference wavelength sampling
    spectra_list = [in_rss[i] for i in range(in_rss._fibers)]
    if new_wave is not None:
        if cpus > 1:
            pool = Pool(cpus)
            threads = []
            for i in range(in_rss._fibers):
                threads.append(pool.apply_async(spectra_list[i].resampleSpec, (new_wave, resample_method, err_sim, replace_error)))

            for i in range(in_rss._fibers):
                spectra_list[i] = threads[i].get()
            pool.close()
            pool.join()
        else:
            for i in range(in_rss._fibers):
                spectra_list[i] = spectra_list[i].resampleSpec(new_wave)
    
    # convolve RSS to reference LSF
    if new_lsf is not None:
        if cpus > 1:
            pool = Pool(cpus)
            threads = []
            for i in range(in_rss._fibers):
                threads.append(pool.apply_async(spectra_list[i].smoothGaussVariable, (new_lsf,)))

            for i in range(in_rss._fibers):
                spectra_list[i] = threads[i].get()
            pool.close()
            pool.join()
        else:
            for i in range(in_rss._fibers):
                spectra_list[i] = spectra_list[i].smoothGaussVariable(new_lsf)
    
    # build RSS
    out_rss = rss.RSS.from_spectra1d(spectra_list=spectra_list)
    
    return out_rss

In [2]:
# get data from lvmdatasimulator
#   * read simulated data in flux units (*_flux.fits)
#   * read simulated data in electrons (*_realization.fits)
#   * read simulated data in electrons without noise (*_no_noise.fits)
#   * calculate flux calibration vector
# write FITS files with:
#   * simulated flux+sky (with noise)
#   * simulated flux (without noise)
#   * simulated sky (without noise)
#   * flux calibration vectors

In [3]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

RSS_INS_PATH = os.path.join(output_path, "rss_ins.pkl")

fetch_example_data(url=LVM_UNAM_URL, name="sim_1d", dest_path=data_path)

[INFO]: example data already exists


In [4]:
sim_name = "DIG_bright_linear_full_900_{}.fits"

# extract observed flux with and without sky
wave = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("flux")), extname="WAVE")
error_flx = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("flux")), extname="ERR")
sci_tot_flx = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("flux")), extname="TOTAL")
sci_obj_flx = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("flux")), extname="TARGET")
sky_mod_flx = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("flux")), extname="SKY")
# extract instrumental flux with and without sky
wave = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("realization")), extname="WAVE")
error_cnt = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("realization")), extname="ERR")
sci_tot_cnt = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("realization")), extname="TOTAL")
sci_obj_cnt = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("realization")), extname="TARGET")
sky_mod_cnt = fits.getdata(os.path.join(data_path, "sim_1d", sim_name.format("realization")), extname="SKY")

In [5]:
# create science RSS
sci_flx_rss = rss.RSS(data=sci_tot_flx, wave=wave, inst_fwhm=np.ones_like(wave)*0.5, error=error_flx)
sci_cnt_rss = rss.RSS(data=sci_tot_cnt, wave=wave, inst_fwhm=np.ones_like(wave)*0.5, error=error_cnt)

# create skies RSSs
NSKY_W, NSKY_E = 59, 60
idx = np.random.choice(np.arange(sci_flx_rss._fibers, dtype=int), NSKY_W+NSKY_E, replace=False)
idx_w, idx_e = idx[:NSKY_W], idx[NSKY_W:]

skyw_flx_rss = rss.RSS(data=sky_mod_flx[idx_w], wave=wave, inst_fwhm=np.ones_like(wave)*0.5)
skyw_cnt_rss = rss.RSS(data=sky_mod_cnt[idx_w], wave=wave, inst_fwhm=np.ones_like(wave)*0.5, error=error_cnt[idx_w])

skye_flx_rss = rss.RSS(data=sky_mod_flx[idx_e], wave=wave, inst_fwhm=np.ones_like(wave)*0.5)
skye_cnt_rss = rss.RSS(data=sky_mod_cnt[idx_e], wave=wave, inst_fwhm=np.ones_like(wave)*0.5, error=error_cnt[idx_e])

# calculate flux calibration vector
flux_cal = skyw_flx_rss.create1DSpec() / skyw_cnt_rss.create1DSpec()

In [8]:
# define new wavelength
dif_wav = 1.0
new_wav = np.arange(wave[0], wave[-1]+dif_wav, dif_wav)
# define differential resolution
res = 4000
new_lsf = new_wav / res
dif_lsf = np.sqrt(new_lsf**2 - 0.5**2)

# downgrade in resolution and resample
if not os.path.isfile(RSS_INS_PATH):
    sci_ins_rss = matchRes(sci_cnt_rss, new_wave=new_wav, new_lsf=dif_lsf, err_sim=10, parallel="auto")
    skyw_ins_rss = matchRes(skyw_cnt_rss, new_wave=new_wav, new_lsf=dif_lsf, err_sim=10, parallel="auto")
    skye_ins_rss = matchRes(skye_cnt_rss, new_wave=new_wav, new_lsf=dif_lsf, err_sim=10, parallel="auto")

    pikle.dump((sci_ins_rss, skyw_ins_rss, skye_ins_rss), open(RSS_INS_PATH, "wb"))
else:
    sci_ins_rss, skyw_ins_rss, skye_ins_rss = pickle.load(open(RSS_INS_PATH, "rb"))

In [9]:
# add noise to resampled spectra to compensate the smoothing effect
noise_sci = np.random.randn(*sci_ins_rss._error.shape) * sci_ins_rss._error
noise_skyw = np.random.randn(*skyw_ins_rss._error.shape) * skyw_ins_rss._error
noise_skye = np.random.randn(*skye_ins_rss._error.shape) * skye_ins_rss._error

sci_ins_rss._data = sci_ins_rss._data + noise_sci
skyw_ins_rss._data = skyw_ins_rss._data + noise_skyw
skye_ins_rss._data = skye_ins_rss._data + noise_skye

In [10]:
sci_ins_rss.writeFitsData(os.path.join(output_path, sim_name.format("sci")))
skyw_ins_rss.writeFitsData(os.path.join(output_path, sim_name.format("skyw")))
skye_ins_rss.writeFitsData(os.path.join(output_path, sim_name.format("skye")))